In [1]:
import datasets

In [ ]:
dsd = datasets.load_dataset("JeanKaddour/minipile")

print(dsd)

In [ ]:
for split in ["train", "validation", "test"]:
    ds = dsd[f"{split}"]
    ds = ds.add_column("id", [f"{split}-{i}" for i in range(len(ds))])
    ds.to_json(
        f"/home/lab/moe/data/raw/minipile/{split}.jsonl", orient="records", lines=True
    )


---

In [ ]:
Total number of parameters:         976,028,672 (1B)
Number of non-embedding parameters: 924,517,376 (924M)
Number of active parameters:        271,385,600 (271M)


In [7]:
import torch as pt
from transformers import OlmoeForCausalLM, AutoTokenizer
DEVICE = pt.device("cpu")
# DEVICE = pt.device("cuda:0")
model_path = "/home/lab/moe/artifacts/olmoe-small-dev"
moe = OlmoeForCausalLM.from_pretrained(model_path, torch_dtype=pt.float16).to(DEVICE)
tok = AutoTokenizer.from_pretrained(model_path)

In [ ]:
inputs = tok("Bitcoin is", return_tensors="pt")
inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
out = moe.generate(**inputs, max_length=32, do_sample=True)
print(tok.decode(out[0]))
